In [1]:
import re
import os
#import sys
import json
from datetime import datetime, timedelta
from bs4 import BeautifulSoup 
import pandas as pd
import re
import emoji
from cbc_paths import file_path_html, file_path_csv

# get current url
file = os.path.abspath('') + '\cbc_stories.json'
with open(file) as json_file:
    cbc_stories = json.load(json_file)

try:
    cbc_url = ([x for x in cbc_stories if x['retrieval_status'] == 'pending'])[0]['url']
    file_name = cbc_url.replace('https://www.cbc.ca/','').replace('news/','').replace('sports/','').replace('/','_') + '.html'
    print(cbc_url)
    print(file_name)
except:
    print('no new urls')
    pass

https://www.cbc.ca/news/business/supply-chain-column-don-pittis-1.6219217
business_supply-chain-column-don-pittis-1.6219217.html


In [2]:
# parse html to get data
# datetime format 2021-09-23T11:59:28.306Z
soup = BeautifulSoup(open(file_path_html + file_name, encoding='utf8').read(), 'html.parser')
publish_date_raw = soup.find('time', class_='timeStamp')['datetime'][:-5]
publish_date = datetime.strptime(str(publish_date_raw), '%Y-%m-%dT%H:%M:%S')
vf_comments = soup.find('div', class_='vf-comments')
vf_comment_threads = soup.find_all('div', class_='vf-comment-container')
vf_usernames = soup.find_all('button', class_='vf-username')
print(cbc_url)
print(file_name)
print(len(vf_comment_threads))

https://www.cbc.ca/news/business/supply-chain-column-don-pittis-1.6219217
business_supply-chain-column-don-pittis-1.6219217.html
2164


In [3]:
# create comment data list of lists
comment_data = []
status = 'open'

# Look for "Commenting is now closed for this story." string on page to record as closed
# if closed_string:
    # status = 'closed'
# else:
    # status = 'open'
    
import unidecode
def replace_unicode(username):
    clean_username = unidecode.unidecode(username)
    return clean_username

for thread in vf_comment_threads:
    # children = data_ids.findChildren()
    # div_data_id = soup.find('div', class_='vf-comment')
    data_id = thread['data-id']
    username = thread.find('button', class_='vf-username').get_text()

    # now_plus_10 = now + datetime.timedelta(minutes = 10)
    # 2021-09-22T23:41:24.974Z
    # 2021-09-23 14:16:22.576000
    # datetime.datetime.strptime(date_time_str, '%Y-%m-%d %H:%M:%S.%f')
    # updated_time = date_time_obj + timedelta(minutes=30)

    # get comment age in minutes (orig could be minutes, hours, days)
    # convert to datetime based on publish datetime
    comment_time_str = thread.find('span', class_='vf-date').get_text().replace('s ago', '')
    comment_time_int = int(re.sub('[^0-9]', '', comment_time_str))
    if 'minute' in comment_time_str:
        elapsed_minutes = comment_time_int
    if 'hour' in comment_time_str:
        elapsed_minutes = comment_time_int * 60
    if 'day' in comment_time_str:
        elapsed_minutes = comment_time_int * 60 * 24
    if 'month' in comment_time_str:
        elapsed_minutes = comment_time_int * 60 * 24 * 30
    comment_time = publish_date - timedelta(minutes=elapsed_minutes)
    #print(comment_time_str, comment_time_int, elapsed_minutes)

    # get comment text
    comment_text_raw = thread.find('span', class_='vf-comment-html-content').get_text()
    if 'Reply to @' in comment_text_raw:
        comment_type = 'reply'
        if len(comment_text_raw.split(':',1)) >=2:
            replied_to_user = comment_text_raw.replace('Reply to @', '').split(':',1)[0].replace('\n','').strip()
            comment_text = comment_text_raw.split(":",1)[1].replace('\n','').strip()
        else:
            replied_to_user = comment_text_raw.replace('Reply to @', '')[0:20].replace('\n','').strip()
            comment_text = comment_text_raw.replace('Reply to @', '').replace('\n','').strip()
    else:
        comment_type = 'parent'
        replied_to_user = ''
        comment_text = comment_text_raw.replace('\n','').strip()

    # append comment data to comment_data list
    comment_data.append((
        data_id, 
        publish_date, 
        comment_time,
        elapsed_minutes,
        replace_unicode(emoji.get_emoji_regexp().sub('', username)).strip(), 
        comment_type, 
        replace_unicode(emoji.get_emoji_regexp().sub('', replied_to_user)).strip(), 
        comment_text, 
        file_name.replace('.html', ''), 
        cbc_url,
        status))

# convert comment_data list to dataframe
df_comment_data = pd.DataFrame(
    list(comment_data), 
    columns=[
    'data_id', 
    'publish time', 
    'comment_time', 
    'minutes',
    'comment_user', 
    'comment_type', 
    'replied_to_user', 
    'comment_text', 
    'file_name',
    'file_url',
    'status'])

# calculate and add elapsed minutes column value
#start_time = df_comment_data['comment_time'].min()
#df_comment_data['minutes'] = ((df_comment_data['comment_time'] - start_time).dt.total_seconds() / 60).astype(int)

df_comment_data.to_csv(file_path_csv + file_name.replace('.html', '.csv'), encoding='utf-8', index=False)
print(cbc_url)
print(file_name)
print(len(vf_comment_threads))

https://www.cbc.ca/news/business/supply-chain-column-don-pittis-1.6219217
business_supply-chain-column-don-pittis-1.6219217.html
2164


In [4]:
df_comment_data.head(5)

,data_id,publish time,comment_time,minutes,comment_user,comment_type,replied_to_user,comment_text,file_name,file_url,status
0,7803700020802,2021-10-25 13:07:54,2021-08-26 13:07:54,86400,Jim Janz,parent,,"Note to Darryl Wier, rename your product ""Knig...",business_supply-chain-column-don-pittis-1.6219217,https://www.cbc.ca/news/business/supply-chain-...,open
1,4860300020678,2021-10-25 13:07:54,2021-08-26 13:07:54,86400,Daniel Toth,parent,,Now could be a good time to get the family tog...,business_supply-chain-column-don-pittis-1.6219217,https://www.cbc.ca/news/business/supply-chain-...,open
2,7438200020887,2021-10-25 13:07:54,2021-08-26 13:07:54,86400,Damon Antonio,parent,,Take the example of the tomato in this article...,business_supply-chain-column-don-pittis-1.6219217,https://www.cbc.ca/news/business/supply-chain-...,open
3,7662500020690,2021-10-25 13:07:54,2021-08-26 13:07:54,86400,James Gudnason,parent,,Creating a miserable business environment thro...,business_supply-chain-column-don-pittis-1.6219217,https://www.cbc.ca/news/business/supply-chain-...,open
4,5049100020919,2021-10-25 13:07:54,2021-08-26 13:07:54,86400,November Jones,parent,,Complaining about the container ship being on ...,business_supply-chain-column-don-pittis-1.6219217,https://www.cbc.ca/news/business/supply-chain-...,open
